In [ ]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from IPython.display import display


In [ ]:
%load_ext autoreload
%autoreload 2
    
from simulation_tools import Population
from simulation_tools import Transformation
from simulation_tools import Metric
from simulation_tools import Experiment


In [ ]:
# Load data:
SYNTHETIC_DATA = './synthetic_data/'

dummy = False

if dummy:

    location_attributes = pd.read_csv(SYNTHETIC_DATA+'location_attributes_dummy.csv')
    location_profiles = pd.read_csv(SYNTHETIC_DATA+'location_profiles_dummy.csv')

else:
    
    location_attributes = pd.read_csv(SYNTHETIC_DATA+'location_attributes_census.csv')
    location_attributes['location_name'] = location_attributes['location_id']

    location_profiles = pd.read_csv(SYNTHETIC_DATA+'location_profiles_census.csv')
    location_profiles['location_name'] = location_profiles['location_id']
    
    # Limit to one state and divide population by a constant to get reasonable sample size.
    state = 'New York'
    constant = 1000
    location_attributes = location_attributes[location_attributes['location_state']==state]
    location_profiles = location_profiles[location_profiles['location_state']==state]
    location_attributes['population'] = (location_attributes['population']/constant).astype(int)
    
print("Loaded data for {} locations ({} people).".format(
    len(location_attributes),location_attributes['population'].sum()
))

display(location_attributes.head())
display(location_profiles.head())


In [ ]:
experiment = Experiment(
    location_attributes, location_profiles, behavior='total_compliance', method='median_person',
    trials=10, show_progress=True, experiment_name=None, random_state=221,
)


In [ ]:
display(experiment.get_results(style='df'))
display(experiment.get_results(style='mean'))
display(experiment.get_results(style='std'))
